In [168]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files={"train": "./data/train.csv", "test": "./data/test.csv"})

Using custom data configuration default-b36b45c6a40b2881


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

d:\miniconda3\envs\snow39\lib\site-packages\datasets\download\streaming_download_manager.py:776: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


Generating test split: 0 examples [00:00, ? examples/s]

d:\miniconda3\envs\snow39\lib\site-packages\datasets\download\streaming_download_manager.py:776: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


Dataset csv downloaded and prepared to C:/Users/леопольд/.cache/huggingface/datasets/csv/default-b36b45c6a40b2881/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [170]:
for partition in ["train", "test"]:
    dataset[partition] = dataset[partition].class_encode_column("rating")

Stringifying the column:   0%|          | 0/25 [00:00<?, ?ba/s]

Casting to class labels:   0%|          | 0/25 [00:00<?, ?ba/s]

Stringifying the column:   0%|          | 0/25 [00:00<?, ?ba/s]

Casting to class labels:   0%|          | 0/25 [00:00<?, ?ba/s]

In [175]:
dataset["train"] = dataset["train"].train_test_split(test_size=0.2, seed=42, stratify_by_column="rating")

Loading cached split indices for dataset at C:\Users\леопольд\.cache\huggingface\datasets\csv\default-b36b45c6a40b2881\0.0.0\6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317\cache-c2a84ea8275b98dd.arrow and C:\Users\леопольд\.cache\huggingface\datasets\csv\default-b36b45c6a40b2881\0.0.0\6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317\cache-db5265c812badd68.arrow


In [180]:
num_labels = dataset["train"]["train"].features["rating"].num_classes
num_labels

8

In [181]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=num_labels)

loading file vocab.txt from cache at C:\Users\леопольд/.cache\huggingface\hub\models--distilbert-base-uncased\snapshots\1c4513b2eedbda136f57676a34eea67aba266e5c\vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at C:\Users\леопольд/.cache\huggingface\hub\models--distilbert-base-uncased\snapshots\1c4513b2eedbda136f57676a34eea67aba266e5c\tokenizer_config.json
loading configuration file config.json from cache at C:\Users\леопольд/.cache\huggingface\hub\models--distilbert-base-uncased\snapshots\1c4513b2eedbda136f57676a34eea67aba266e5c\config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_hea

In [186]:
def tokenize(x, tokenizer):
    x.update(tokenizer(x["text"]))
    return x

tokenized_dataset = dataset["train"].map(lambda x: tokenize(x, tokenizer))

  0%|          | 0/20000 [00:00<?, ?ex/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (584 > 512). Running this sequence through the model will result in indexing errors


  0%|          | 0/5000 [00:00<?, ?ex/s]

In [216]:
import torch
from collections import defaultdict

class Collator:
    MAX_SEQUENCE_LENGTH = 512
    def __init__(self, padded_columns, device='cuda'):
        self.padded_columns = set(padded_columns)
        self.device = device
        self.label_column = "labels"

    def __call__(self, batch):
        padded_batch = defaultdict(list)
        for example in batch:
            for key, tensor in example.items():
                padded_batch[key].append(tensor)
                
        for key, val in padded_batch.items():
            if key == self.label_column:
                padded_batch[key] = torch.tensor(val).to(self.device)
            elif key in self.padded_columns:
                padded_batch[key] = torch.nn.utils.rnn.pad_sequence(val, batch_first=True)[:, :self.MAX_SEQUENCE_LENGTH].to(self.device)
        return dict(padded_batch)


In [192]:
for partition in ["train", "test"]:
    tokenized_dataset[partition] = tokenized_dataset[partition].rename_columns({"label": "claz", "rating": "labels"},)

In [209]:
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'], output_all_columns=True)

In [210]:
dl = torch.utils.data.DataLoader(tokenized_dataset["test"], batch_size=4, collate_fn=Collator(['input_ids', 'attention_mask']))
next(iter(dl))

defaultdict(list,
            {'labels': tensor([0, 7, 1, 0], device='cuda:0'),
             'input_ids': tensor([[  101, 10468,  2023,  ...,  3430,  1012,   102],
                     [  101,  2446,  2797,  ...,     0,     0,     0],
                     [  101,  1045,  2034,  ...,     0,     0,     0],
                     [  101,  2023,  2051,  ...,     0,     0,     0]], device='cuda:0'),
             'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
                     [1, 1, 1,  ..., 0, 0, 0],
                     [1, 1, 1,  ..., 0, 0, 0],
                     [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0'),
             'id': [11661, 9336, 4111, 4931],
             'text': ["Basically this is an overlong |  unfunny |  action/comedy. First of all I'd like to say that I did enjoy the Wayans brother Scary Movie (1) and the sequel had it's moments. Unfortunately white chicks doesn't even deliver HALF the laughs. <br /><br />The humour in it is absolutely crude. If you like burping | 

In [221]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def get_rating_from_labels(labels):
    return np.array(
        list(
            map(int, tokenized_dataset["train"].features["labels"].int2str(labels))
        )
    )

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    sentiment_pred = np.where(get_rating_from_labels(predictions) <= 4, 0, 1)
    sentiment_labels = np.where(get_rating_from_labels(labels) <= 4, 0, 1)

    return {
        "ratings_accuracy": metric.compute(predictions=predictions, references=labels)["accuracy"],
        "sentiment_accuracy": metric.compute(predictions=sentiment_pred, references=sentiment_labels)["accuracy"],
    }

In [218]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir="sentiment_trainer",
    evaluation_strategy="steps",
    eval_steps=1000,
    dataloader_pin_memory=False
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=Collator(padded_columns=["input_ids", "attention_mask"]),
    compute_metrics=compute_metrics
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [219]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: id, claz, text. If id, claz, text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 20000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 7500
  Number of trainable parameters = 66959624


  0%|          | 0/7500 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [225]:
get_rating_from_labels([1])

array([10])

In [226]:
trainer.evaluate(tokenized_dataset["test"])

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: id, claz, text. If id, claz, text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 8


{'eval_loss': 2.007974624633789, 'eval_ratings_accuracy': 0.3216, 'eval_sentiment_accuracy': 0.7516, 'eval_runtime': 84.0163, 'eval_samples_per_second': 59.512, 'eval_steps_per_second': 7.439, 'epoch': 0.01}


{'eval_loss': 2.007974624633789,
 'eval_ratings_accuracy': 0.3216,
 'eval_sentiment_accuracy': 0.7516,
 'eval_runtime': 84.0163,
 'eval_samples_per_second': 59.512,
 'eval_steps_per_second': 7.439,
 'epoch': 0.01}